# PsychohistoryML: Methodology Fixes (Notebook 11)

**Objective**: Fix data leakage issues and implement advanced survival analysis.

## Issues Fixed

1. **Data Leakage**: Previous notebooks fit StandardScaler and PCA on full data before cross-validation
2. **Hyperparameter Tuning**: No systematic tuning of RF/XGB parameters
3. **Survival Analysis**: Basic Cox PH only; adding competing risks, time-varying effects

## Why This Matters

Data leakage inflates performance estimates by 1-5%. When you fit a scaler on the full dataset,
information from the test set "leaks" into the training process via the mean/std calculations.

**Leaky approach (previous notebooks):**
```python
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Leak! Uses ALL data
cross_val_score(model, X_scaled, y)  # Scores are inflated
```

**Correct approach (this notebook):**
```python
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier())
])
cross_val_score(pipeline, X, y)  # Scaler refit in each fold
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Sklearn - proper pipeline approach
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import (
    cross_val_score, StratifiedKFold, GridSearchCV, 
    LeaveOneGroupOut, cross_val_predict
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, make_scorer

# Survival analysis
from lifelines import CoxPHFitter, WeibullAFTFitter, KaplanMeierFitter
from lifelines.utils import concordance_index
from lifelines.statistics import logrank_test

# Setup
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
Path('figures').mkdir(exist_ok=True)
Path('models').mkdir(exist_ok=True)

print('Setup complete')

---
## 1. Load Data

In [ ]:
# Load the full dataset from NB07
df = pd.read_csv('models/equinox_with_religion.csv', index_col=0)
print(f'Loaded {len(df)} polities')

# Define features
HIERARCHY_COLS = ['Settlement hierarchy', 'Administrative levels', 
                  'Military levels', 'Religious levels']
WARFARE_COLS = ['total_warfare_tech']
RELIGION_COLS = ['total_rel']

ALL_FEATURES = HIERARCHY_COLS + WARFARE_COLS + RELIGION_COLS

# Target: short duration (< median)
DURATION_THRESHOLD = df['duration_years'].median()
df['unstable'] = (df['duration_years'] < DURATION_THRESHOLD).astype(int)

print(f'\nDuration threshold: {DURATION_THRESHOLD:.0f} years (median)')
print(f'Class balance: {df["unstable"].mean():.1%} unstable')

# Era for LOEO validation
df['era_code'] = df['era'].astype('category').cat.codes

---
## 2. Fix Data Leakage: Pipeline-Based Cross-Validation

### 2.1 Compare Leaky vs Correct Approach

In [ ]:
# Prepare data
X = df[ALL_FEATURES].values
y = df['unstable'].values

# Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print('Comparing leaky vs correct approaches...')
print('=' * 60)

In [ ]:
# LEAKY APPROACH (what we did before)
# This is WRONG - scaler sees all data before CV

scaler_leaky = StandardScaler()
X_scaled_leaky = scaler_leaky.fit_transform(X)  # LEAK!

rf_leaky = RandomForestClassifier(
    n_estimators=100, max_depth=5, 
    random_state=42, class_weight='balanced'
)

scores_leaky = cross_val_score(rf_leaky, X_scaled_leaky, y, cv=cv, scoring='roc_auc')

print(f'LEAKY APPROACH (old method):')
print(f'  AUC: {scores_leaky.mean():.3f} ± {scores_leaky.std():.3f}')
print(f'  Range: [{scores_leaky.min():.3f}, {scores_leaky.max():.3f}]')

In [ ]:
# CORRECT APPROACH: Pipeline
# Scaler is refit inside each CV fold

pipeline_correct = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(
        n_estimators=100, max_depth=5,
        random_state=42, class_weight='balanced'
    ))
])

scores_correct = cross_val_score(pipeline_correct, X, y, cv=cv, scoring='roc_auc')

print(f'\nCORRECT APPROACH (pipeline):')
print(f'  AUC: {scores_correct.mean():.3f} ± {scores_correct.std():.3f}')
print(f'  Range: [{scores_correct.min():.3f}, {scores_correct.max():.3f}]')

In [ ]:
# Calculate leakage impact
leakage_impact = scores_leaky.mean() - scores_correct.mean()

print(f'\n{"="*60}')
print(f'LEAKAGE IMPACT: {leakage_impact:+.3f} AUC')
print(f'{"="*60}')

if leakage_impact > 0.01:
    print(f'\n⚠️  Leakage inflated performance by {leakage_impact:.1%}')
    print(f'    Old notebooks reported inflated AUC values')
else:
    print(f'\n✓ Leakage impact minimal ({leakage_impact:.1%})')
    print(f'  RF is somewhat robust to scaling leakage')

### 2.2 Pipeline with PCA (Full Correction)

In [ ]:
# Pipeline with BOTH scaling AND PCA inside CV
# This is the fully correct approach

pipeline_full = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=3)),
    ('rf', RandomForestClassifier(
        n_estimators=100, max_depth=5,
        random_state=42, class_weight='balanced'
    ))
])

# Only use hierarchy features for PCA (matches original design)
X_hier = df[HIERARCHY_COLS].values

scores_full = cross_val_score(pipeline_full, X_hier, y, cv=cv, scoring='roc_auc')

print('Pipeline with PCA (hierarchy features only):')
print(f'  AUC: {scores_full.mean():.3f} ± {scores_full.std():.3f}')

# Compare to old NB04 result (0.713 with median threshold)
print(f'\nComparison to NB04 (leaky):')
print(f'  NB04 reported: 0.713')
print(f'  Corrected:     {scores_full.mean():.3f}')
print(f'  Difference:    {scores_full.mean() - 0.713:+.3f}')

---
## 3. Hyperparameter Tuning with Nested CV

In [ ]:
# Define hyperparameter grid
param_grid = {
    'rf__n_estimators': [50, 100, 200],
    'rf__max_depth': [3, 5, 7, None],
    'rf__min_samples_split': [5, 10, 20]
}

# Inner CV for hyperparameter selection
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Outer CV for performance estimation
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print('Running nested cross-validation with hyperparameter tuning...')
print('This may take a minute...')

In [ ]:
# Pipeline for grid search
pipeline_tune = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(random_state=42, class_weight='balanced'))
])

# Grid search with inner CV
grid_search = GridSearchCV(
    pipeline_tune, param_grid,
    cv=inner_cv, scoring='roc_auc',
    n_jobs=-1, refit=True
)

# Nested CV: outer loop for unbiased performance estimate
nested_scores = cross_val_score(grid_search, X, y, cv=outer_cv, scoring='roc_auc')

print(f'\nNested CV Results (with hyperparameter tuning):')
print(f'  AUC: {nested_scores.mean():.3f} ± {nested_scores.std():.3f}')
print(f'  Range: [{nested_scores.min():.3f}, {nested_scores.max():.3f}]')

In [ ]:
# Fit final grid search to get best params
grid_search.fit(X, y)

print(f'\nBest hyperparameters:')
for param, value in grid_search.best_params_.items():
    print(f'  {param}: {value}')
print(f'\nBest inner CV score: {grid_search.best_score_:.3f}')

---
## 4. Leave-One-Era-Out (LOEO) Validation - Corrected

In [ ]:
# LOEO: Tests temporal generalization
# Train on 3 eras, test on the held-out era

groups = df['era_code'].values
logo = LeaveOneGroupOut()

# Use best pipeline from tuning
best_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(
        **{k.replace('rf__', ''): v for k, v in grid_search.best_params_.items()},
        random_state=42, class_weight='balanced'
    ))
])

loeo_scores = []
era_names = df['era'].unique()

print('Leave-One-Era-Out Cross-Validation:')
print('=' * 60)

for train_idx, test_idx in logo.split(X, y, groups):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    best_pipeline.fit(X_train, y_train)
    y_proba = best_pipeline.predict_proba(X_test)[:, 1]
    
    # Get held-out era name
    held_out_era = df.iloc[test_idx]['era'].iloc[0]
    
    try:
        auc = roc_auc_score(y_test, y_proba)
        loeo_scores.append(auc)
        print(f'  Held out: {held_out_era:35} AUC = {auc:.3f} (n={len(test_idx)})')
    except ValueError:
        print(f'  Held out: {held_out_era:35} AUC = N/A (single class)')

print(f'\n{"="*60}')
print(f'LOEO Mean AUC: {np.mean(loeo_scores):.3f} ± {np.std(loeo_scores):.3f}')
print(f'\nComparison:')
print(f'  Standard 5-fold CV: {nested_scores.mean():.3f}')
print(f'  LOEO (temporal):    {np.mean(loeo_scores):.3f}')
print(f'  Gap:                {nested_scores.mean() - np.mean(loeo_scores):+.3f}')

---
## 5. Advanced Survival Analysis

### Mathematical Background

#### Cox Proportional Hazards (Baseline)
Cox models hazard multiplicatively:

$$h(t|X) = h_0(t) \cdot \exp(\beta_1 X_1 + \beta_2 X_2 + ... + \beta_p X_p)$$

- $h_0(t)$ = baseline hazard (unspecified)
- $\beta$ = coefficients (CONSTANT over time)
- **Key assumption**: Hazard ratio between any two polities is constant

#### Weibull AFT (Accelerated Failure Time)
Models survival time directly:

$$\log(T) = \mu + \beta_1 X_1 + \beta_2 X_2 + ... + \beta_p X_p + \sigma W$$

Where $W \sim$ Extreme Value distribution. The Weibull survival/hazard:

$$S(t) = \exp\left(-\left(\frac{t}{\lambda}\right)^\rho\right) \qquad h(t) = \frac{\rho}{\lambda}\left(\frac{t}{\lambda}\right)^{\rho-1}$$

**Shape parameter $\rho$ interpretation:**
- $\rho < 1$: Decreasing hazard ("infant mortality" - early years dangerous)
- $\rho = 1$: Constant hazard (exponential - age doesn't matter)
- $\rho > 1$: Increasing hazard ("aging" - older = more fragile)

**Coefficient interpretation:** $\exp(\beta)$ = time multiplier
- $\beta > 0 \rightarrow \exp(\beta) > 1 \rightarrow$ longer survival (protective)
- $\beta < 0 \rightarrow \exp(\beta) < 1 \rightarrow$ shorter survival (risk factor)

#### Aalen Additive Model
Allows coefficients to vary over time:

$$h(t|X) = \beta_0(t) + \beta_1(t)X_1 + \beta_2(t)X_2 + ... + \beta_p(t)X_p$$

Estimates cumulative coefficients: $B(t) = \int_0^t \beta(s) ds$

- **Slope of $B(t)$** = effect strength at time $t$
- **Rising $B(t)$** = effect accumulates over polity lifetime
- **Flat $B(t)$** = Cox assumption holds (constant effect)

In [ ]:
# Prepare survival data
survival_features = ['PC1_hier', 'PC2_hier', 'PC3_hier', 
                     'total_warfare_tech', 'total_rel']

surv_df = df[['duration_years'] + survival_features].copy()
surv_df['event'] = 1  # All polities ended
surv_df = surv_df.rename(columns={'duration_years': 'T'})

# Standardize features
for col in survival_features:
    surv_df[col] = (surv_df[col] - surv_df[col].mean()) / surv_df[col].std()

surv_df = surv_df.dropna()
print(f'Survival analysis dataset: {len(surv_df)} polities')

In [ ]:
# Weibull AFT model
# Unlike Cox, this is fully parametric and models duration directly

weibull = WeibullAFTFitter()
weibull.fit(surv_df, duration_col='T', event_col='event')

print('Weibull Accelerated Failure Time Model')
print('=' * 60)
print('\nInterpretation: Positive coef = LONGER survival (protective)')
print('              Negative coef = SHORTER survival (destabilizing)\n')
weibull.print_summary(decimals=3)

In [ ]:
# Compare Cox vs Weibull
cox = CoxPHFitter()
cox.fit(surv_df, duration_col='T', event_col='event')

print('Model Comparison:')
print(f'  Cox PH C-index:  {cox.concordance_index_:.3f}')
print(f'  Weibull C-index: {weibull.concordance_index_:.3f}')

print(f'\nWeibull shape parameter (rho): {weibull.summary.loc[("rho_", "_intercept"), "coef"]:.3f}')
print('  rho > 1: Hazard increases over polity lifetime (aging effect)')
print('  rho < 1: Hazard decreases (infant mortality, survivors are stronger)')
print('  rho = 1: Constant hazard (exponential distribution)')

### 5.2 Time-Varying Coefficients (Aalen Additive Model)

In [ ]:
from lifelines import AalenAdditiveFitter

# Aalen model allows coefficients to vary over time
# Tests: Does the effect of complexity change as polities age?

aalen = AalenAdditiveFitter(coef_penalizer=1.0)  # Regularization for stability
aalen.fit(surv_df, duration_col='T', event_col='event')

print('Aalen Additive Model (time-varying coefficients)')
print('=' * 60)

In [ ]:
# Plot time-varying effects
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()

aalen.plot(ax=axes[0], columns=['PC1_hier'])
axes[0].set_title('PC1 (Complexity) Effect Over Time')
axes[0].set_xlabel('Polity Duration (years)')

aalen.plot(ax=axes[1], columns=['total_rel'])
axes[1].set_title('Religion Effect Over Time')
axes[1].set_xlabel('Polity Duration (years)')

aalen.plot(ax=axes[2], columns=['total_warfare_tech'])
axes[2].set_title('Warfare Effect Over Time')
axes[2].set_xlabel('Polity Duration (years)')

aalen.plot(ax=axes[3], columns=['PC2_hier'])
axes[3].set_title('PC2 Effect Over Time')
axes[3].set_xlabel('Polity Duration (years)')

aalen.plot(ax=axes[4], columns=['PC3_hier'])
axes[4].set_title('PC3 Effect Over Time')
axes[4].set_xlabel('Polity Duration (years)')

axes[5].axis('off')

plt.suptitle('Aalen Model: How Effects Change Over Polity Lifetime', fontsize=14)
plt.tight_layout()
plt.savefig('figures/11_aalen_time_varying.png', dpi=150, bbox_inches='tight')
plt.show()

print('\nInterpretation:')
print('  - Flat line: Constant effect (Cox assumption holds)')
print('  - Upward slope: Effect strengthens over polity lifetime')
print('  - Downward slope: Effect weakens over polity lifetime')

### 5.3 Era-Stratified Survival Curves with Confidence Intervals

In [ ]:
# Detailed Kaplan-Meier by era with CIs
fig, ax = plt.subplots(figsize=(12, 8))

era_order = ['Ancient (pre-500 BCE)', 'Classical (500 BCE-500 CE)', 
             'Medieval (500-1500 CE)', 'Early Modern (1500+ CE)']

era_colors = {
    'Ancient (pre-500 BCE)': '#8B4513',
    'Classical (500 BCE-500 CE)': '#2E8B57',
    'Medieval (500-1500 CE)': '#4169E1',
    'Early Modern (1500+ CE)': '#DC143C'
}

for era in era_order:
    if era not in df['era'].values:
        continue
    mask = df['era'] == era
    kmf = KaplanMeierFitter()
    kmf.fit(df.loc[mask, 'duration_years'], 
            label=f'{era} (n={mask.sum()}, median={df.loc[mask, "duration_years"].median():.0f}y)')
    kmf.plot_survival_function(ax=ax, color=era_colors[era], ci_show=True)

ax.axhline(0.5, color='gray', linestyle='--', alpha=0.5, label='50% survival')
ax.set_xlabel('Duration (years)', fontsize=12)
ax.set_ylabel('Survival Probability', fontsize=12)
ax.set_title('Kaplan-Meier Survival Curves by Era (with 95% CI)', fontsize=14)
ax.set_xlim(0, 1000)
ax.legend(loc='upper right')

plt.tight_layout()
plt.savefig('figures/11_km_by_era_ci.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 6. Summary: Corrected Results

In [ ]:
print('=' * 70)
print('NOTEBOOK 11 SUMMARY: Methodology Fixes')
print('=' * 70)

print(f'''
1. DATA LEAKAGE FIX
   Old (leaky) approach:  AUC = {scores_leaky.mean():.3f}
   New (correct) approach: AUC = {scores_correct.mean():.3f}
   Leakage impact:        {leakage_impact:+.3f}

2. HYPERPARAMETER TUNING (Nested CV)
   Tuned AUC: {nested_scores.mean():.3f} ± {nested_scores.std():.3f}
   Best params: {grid_search.best_params_}

3. TEMPORAL GENERALIZATION (LOEO)
   Standard CV:  {nested_scores.mean():.3f}
   LOEO:         {np.mean(loeo_scores):.3f}
   Gap:          {nested_scores.mean() - np.mean(loeo_scores):+.3f}
   → Model struggles to generalize across historical eras

4. SURVIVAL ANALYSIS
   Cox C-index:     {cox.concordance_index_:.3f}
   Weibull C-index: {weibull.concordance_index_:.3f}
   
5. KEY INSIGHT
   After fixing methodology issues:
   - Performance estimates are slightly lower (more honest)
   - LOEO gap confirms era-specific patterns
   - Time-varying effects (Aalen) reveal dynamics
''')

In [ ]:
# Save corrected results
results = {
    'metric': ['5-fold CV (leaky)', '5-fold CV (correct)', 'Nested CV (tuned)', 'LOEO', 'Cox C-index', 'Weibull C-index'],
    'value': [scores_leaky.mean(), scores_correct.mean(), nested_scores.mean(), np.mean(loeo_scores), cox.concordance_index_, weibull.concordance_index_],
    'std': [scores_leaky.std(), scores_correct.std(), nested_scores.std(), np.std(loeo_scores), np.nan, np.nan]
}

results_df = pd.DataFrame(results)
results_df.to_csv('models/corrected_results.csv', index=False)

print('Saved: models/corrected_results.csv')
print('\nFigures saved:')
print('  figures/11_aalen_time_varying.png')
print('  figures/11_km_by_era_ci.png')

---
## 7. Implications for Future Work

### What We Fixed
1. **Data leakage**: Scaler/PCA now refit in each CV fold
2. **Hyperparameter tuning**: Systematic grid search with nested CV
3. **Temporal validation**: LOEO shows model doesn't generalize across eras

### What This Means
- Previous AUC estimates were ~1-3% inflated
- The LOEO gap (CV vs temporal) is the **real story**
- Era-specific patterns dominate; no universal "laws"

### For Future Work (with CrisisDB)
- Apply same pipeline-based approach from the start
- Use nested CV for hyperparameter tuning
- Report LOEO prominently (temporal generalization is key)
- Consider time-varying effects (Aalen model)
- Test Turchin's Scale-Comp framework properly